In [1]:
import src
import pymongo
from pymongo.collection import Collection
from sklearn.preprocessing import StandardScaler
import numpy as np
from rich import print
import pickle
from typing import List, Dict, Tuple

# connect to the database
MLA = src.models.MLA(r"C:\Users\ariel\Documents\Courses\Tesi\Code\config.yaml")

train_data = MLA.col_train.find_one({'_id': 'training set'})

if train_data is None:  # if the training set is empty, initialize it with the oldest snapshot
    snap = MLA.col_features.find().sort('timestamp',pymongo.ASCENDING).limit(1)[0]  # get the oldest snapshot
    snap['_id']='training set'                                                      # rename it for initializing the training set
    MLA.col_train.insert_one(snap)                                                  # insert it in the training set   
    print("Training set initialized") 
else:                   # append healty documents to the dataset
    cursor = MLA.col_features.find().sort('timestamp',pymongo.ASCENDING)  # get the oldest snapshot
    for snap in cursor:
        if isinstance(train_data['timestamp'],list):                     # if the training set is a list, pass
            pass
        else:                                                            # convert everityng to list
            train_data['timestamp'] = [train_data['timestamp']]
            for sensor in MLA.sensors:
                for feature in train_data[sensor].keys():
                    train_data[sensor][feature] = [train_data[sensor][feature]]          
        train_data['timestamp'].append(snap['timestamp'])                  # append the timestamp
        for sensor in MLA.sensors:
            for feature in train_data[sensor].keys():
                train_data[sensor][feature].append(snap[sensor][feature])  # append the sensor data
        MLA.col_features.delete_one({'_id': snap['_id']})                  # delete the snapshot from the features collection
   
    MLA.col_train.replace_one({'_id': 'training set'}, train_data)         # replace the training set with the updated one                         


IndexError: no such item for Cursor instance

In [52]:
# now this script scales the data
train_data = MLA.col_train.find_one({'_id': 'training set'})             # get the training set
if train_data is None:
    raise Exception('Training set not initialized')
train_data_scaled = train_data.copy()                                   # copy the training set
train_data_scaled['_id'] = 'training set scaled'                        # rename it
# scale the features
StdScaler: Dict[str, StandardScaler]= {}
for sensor in MLA.sensors:
    StdScaler[sensor] = StandardScaler()
    data = np.array(list(train_data[sensor].values()))      # the scaler wants the data in the form (n_samples, n_features)
    StdScaler[sensor].fit(data.transpose())                              # fit the scaler
    data_scaled = StdScaler[sensor].transform(data.transpose()).transpose()         # the scaler returns the data in the form (n_features, n_samples)
    data_scaled = data_scaled.tolist()                                  # convert the data to list    

    for indx, feature in enumerate(train_data_scaled[sensor].keys()):
        train_data_scaled[sensor][feature] = data_scaled[indx]         # the scaler returns the data in the form (n_features, n_samples)

# save the scaled data
MLA.col_train.delete_many({"_id": 'training set scaled'}) 
MLA.col_train.insert_one(train_data_scaled) 

# save the scaler
pickled_data = pickle.dumps(StdScaler)
try:
    MLA.col_train.insert_one({'_id': 'StandardScaler_pickled', 'data': pickled_data})
except:
    MLA.col_train.replace_one({'_id': 'StandardScaler_pickled'}, {'_id': 'StandardScaler_pickled', 'data': pickled_data})
retrieved_data: Collection | None = MLA.col_train.find_one({'_id': 'StandardScaler_pickled'})
if retrieved_data is None:
    raise Exception('Scaler not found')
else:
    StdScaler_retrieve: Dict[str, StandardScaler] = pickle.loads(retrieved_data['data'])

# check if the scaler perform correctly    
for sensor in MLA.sensors:
    data = np.array(list(train_data[sensor].values()))      # the scaler wants the data in the form (n_samples, n_features)
    data_scaled = StdScaler[sensor].transform(data.transpose()).transpose()         # the scaler returns the data in the form (n_features, n_samples)
    data_scaled_retrieve = StdScaler_retrieve[sensor].transform(data.transpose()).transpose()         # the scaler returns the data in the form (n_features, n_samples)
    if not data_scaled.tolist() == data_scaled_retrieve.tolist():
        raise Exception('Scaler not saved correctly')
    else:
        print(f"Scaler for sensor '{sensor}' saved correctly")
    


Scaler for sensor 'Bearing 1 x' saved correctly

Scaler for sensor 'Bearing 1 y' saved correctly

Scaler for sensor 'Bearing 2 x' saved correctly